In [1]:
#final one
import cv2
import torch
from torchvision import transforms, models 
from PIL import Image
import torch.nn as nn
import os
import numpy as np
import librosa
import pandas as pd
import wave
import pyaudio
import time
from collections import Counter
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
features_df = pd.read_csv('features.csv')

# Extract features and corresponding labels
X = features_df.iloc[:, :-1].values  # features 
y = features_df.iloc[:, -1].values  # labels 

# Initialize the LabelEncoder
label_encoder = LabelEncoder()
# Standardize the features
scaler = StandardScaler()
# Encode the labels
y = label_encoder.fit_transform(y)

# Convert to PyTorch tensors
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.int64)

# Split the dataset into training, validation, and testing sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [2]:
import cv2
import threading
import pyaudio
import time
import wave
import torch
import numpy as np
import librosa
from collections import deque, Counter
from torchvision import transforms, models
from torch import nn
import pandas as pd
from sklearn.preprocessing import StandardScaler
from PIL import Image  


# Constants
FRAME_SKIP = 3
FRAME_PROCESS_INTERVAL = 10
SAMPLE_RATE = 30
input_size = 62
hidden_size = 64
num_classes = 2
sample_rate = 44100

# Image transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

class CustomVGG16(nn.Module):
    def __init__(self, num_classes=6, dropout_rate=0.7):
        super(CustomVGG16, self).__init__()

        model = models.vgg16(pretrained=True)
        for param in model.features[8:].parameters():
            param.requires_grad = True

        self.features = nn.Sequential(
            *list(model.features.children())[:24],
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            *list(model.features.children())[24:],
        )
        self.avgpool = model.avgpool
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(dropout_rate),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(dropout_rate),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
# Class names
class_names = ['Automotive_commercial', 'Entertainment_commercial', 'Food_commercial',
               'Healthcare_commercial', 'Insurance_commercial', 'Technology_Electronics_commercial']

# Initialize and load the VGG16 model
model = CustomVGG16(num_classes=6)
model.load_state_dict(torch.load('best_model_checkpoint.pth', map_location='cpu'))
model.to('cpu')
model.eval()






/home/waqar/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/waqar/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CustomVGG16(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, di

In [3]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out

input_size = 62  
hidden_size = 64
num_classes = 2 

model2 = RNN(input_size, hidden_size, num_classes)
model2.load_state_dict(torch.load('rnn_model.pth', map_location='cpu'))
model2.to('cpu')
model2.eval()



RNN(
  (rnn): RNN(62, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=2, bias=True)
)

In [4]:
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')

        # MFCC (Mel-frequency cepstral coefficients)
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)

        # Chroma feature
        chroma_stft = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)

        # Spectral contrast
        spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sample_rate).T, axis=0)

        # Spectral centroid
        spectral_centroids = np.mean(librosa.feature.spectral_centroid(y=audio, sr=sample_rate).T, axis=0)

        # Zero-crossing rate
        zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y=audio).T, axis=0)

        # Spectral rolloff
        spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=audio, sr=sample_rate).T, axis=0)

        # Combine all features into a 1D array
        features = np.hstack([mfccs_processed, chroma_stft, spectral_contrast, spectral_centroids, zero_crossing_rate, spectral_rolloff])

        return features
    except Exception as e:
        print(f"Error encountered while parsing file: {file_name}")
        return None



In [5]:
def slide_and_record_additional_audio(existing_frames, seconds_to_record=10):
    sample_rate = 44100
    frames_to_discard = int(sample_rate / 1024 * seconds_to_record)

    # Discard the first 'seconds_to_record' worth of frames
    existing_frames = existing_frames[frames_to_discard:]

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)

    # Record new audio to append
    for _ in range(0, frames_to_discard):
        data = stream.read(1024)
        existing_frames.append(data)

    stream.stop_stream()
    stream.close()
    p.terminate()

    return existing_frames  # Return the updated frames


def classify_frames_using_vgg(frame):
    orig_frame = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame)
    transformed_image = transform(pil_image)
    batch_t = torch.unsqueeze(transformed_image, 0).to('cpu')

    with torch.no_grad():
        out = model(batch_t)

    _, predicted = torch.max(out, 1)
    label = class_names[predicted.item()]
    cv2.putText(orig_frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    return orig_frame 




In [6]:
FRAME_PROCESS_DURATION = 5 

def record_and_classify_audio(video_feed):
    audio_duration = 60  # seconds
    sample_rate = 44100
    audio_frames = []

    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt16, channels=1, rate=sample_rate, input=True, frames_per_buffer=1024)

    global commercial_detected
    commercial_detected = False 

    
    print("Starting audio recording and classification...")
    while True:
        # Record audio for the specified duration
        if len(audio_frames) < int(sample_rate / 1024 * audio_duration):
            data = stream.read(1024)
            audio_frames.append(data)
        else:
            # Save the recorded audio to a WAV file
            output_audio_file = "recorded_audio.wav"
            with wave.open(output_audio_file, 'wb') as wf:
                wf.setnchannels(1)
                wf.setsampwidth(p.get_sample_size(pyaudio.paInt16))
                wf.setframerate(sample_rate)
                wf.writeframes(b''.join(audio_frames))

            # Extract features from the audio file
            recorded_features = extract_features(output_audio_file)
            if recorded_features is not None:
                recorded_df = pd.DataFrame([recorded_features])
                scaler = StandardScaler().fit(recorded_df)
                recorded_df = scaler.transform(recorded_df)

                model2.eval()
                recorded_tensor = torch.tensor(recorded_df, dtype=torch.float32)
                with torch.no_grad():

                    predicted_output = torch.sigmoid(model2(recorded_tensor.unsqueeze(1))[:, -1])
                    
                    predicted_class = (predicted_output).float()
                # Run VGG for 10 seconds if class is 0.0
                if predicted_class.item() == 0.0:
                    print("Audio classified as commercial. Starting VGG16 model for 10 seconds.")
                    print(predicted_class.item())
                    start_time = time.time()  # Save the current time
                    commercial_detected = True  # Set flag to true
                else:
                    print("Audio classified as non-commercial. Stopping VGG16 model.")
                    commercial_detected = False  # Set flag to false


            # Slide audio window
            audio_frames = slide_and_record_additional_audio(audio_frames)

        ret, frame = video_feed.read()
        if ret:
            # Check if a commercial is detected
            if commercial_detected:
                frame = classify_frames_using_vgg(frame)  # This function returns a modified frame with label

                # Check if we have processed the frames for more than 5 seconds
                if time.time() - start_time > FRAME_PROCESS_DURATION:
                    commercial_detected = False
                    
            # Check if the frame is valid before displaying
            if frame is not None and isinstance(frame, np.ndarray):
                cv2.imshow('Live Feed with Classifications', frame)
            
            # Exit loop when 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    stream.stop_stream()
    stream.close()
    p.terminate()



In [ ]:

# Thread to display and process frames
def display_and_process_frames():
    global commercial_detected  # Include this to consider RNN model output

    cap = cv2.VideoCapture(2)
    
    while True:
        ret, frame = cap.read()
        with lock:
            if commercial_detected:  # Check this flag here
                frame = classify_frames_using_vgg(frame)

        
        cv2.imshow('Live Feed with Classifications', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Main Function
if __name__ == "__main__":
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open video capture.")
        exit()

    audio_thread = threading.Thread(target=record_and_classify_audio, args=(cap,))
    video_thread = threading.Thread(target=display_and_process_frames)
    
    audio_thread.start()
    video_thread.start()
    
    audio_thread.join()
    video_thread.join()
    
    cap.release()
    cv2.destroyAllWindows()

[ WARN:0@5.497] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@5.497] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
ALSA lib pcm_dsnoop.c:601:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1032:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib pcm_oss.c:397:(_snd_pcm_oss_open) Cannot open device /dev/dsp
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib confmisc.c:160:(snd_config_get_card) Invalid field card
ALSA lib pcm_usb_stream.c:482:(_snd_pcm_usb_stream_open) Invalid card 'card'
ALSA lib pcm_dm

Error: Could not open video capture.
Starting audio recording and classification...


/home/waqar/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


Audio classified as non-commercial. Stopping VGG16 model.


/home/waqar/.local/lib/python3.10/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)


Audio classified as non-commercial. Stopping VGG16 model.
